# Prophet - Predecir el valor de Bitcoin


Prophet es una de las librerías más avanzadas para predecir series temporales desarrollada por Facebook. Te enseñaremos a como entrenar un modelo con Prophet, a optimizarlo y a utilizarlo para realizar predicciones futuras. En este ejercicio vamos a practicar a predecir el valor de Bitcoin, una criptomoneda. Es la criptomoneda que le ha marcado el camino a todas las demás que llegaron después utilizando su tecnología.  

<hr/>
<div class="alert alert-success alertsuccess" style="margin-top: 20px">
[Tip]: Para ejecutar el código de Python en la celda de código a continuación, haz clic en la celda para seleccionarla y presiona <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>
<hr/>


In [2]:
#Dependencias de Prophet
!pip install pystan==2.19.1.1 --quiet
!pip install cmdstanpy --quiet
# Instalamos el modelo predictivo Prophet
!pip install fbprophet --quiet
# De aquí vamos a descargar los datos
!pip install yfinance --quiet

In [3]:
#Importamos las dependencias
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

<h3 id="version">Cargar histórico de datos</h3>


<p>
    Para descargarnos la serie temporal de BTC lo vamos a hacer desde Yahoo Finance. En concreto vamos a descargar el histórico desde 2016 hasta la actualidad.
</p>



In [14]:
# Fecha de inicio es un string 2016-01-01 y fecha de fin (hoy) en formato texto '%Y-%m-%d'
today ='2023-02-15' 
start_date = '2016-01-01'
# Descargar el dataframe
df = yf.download('GC=F',start_date, today).reset_index()
df.tail()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
1786,2023-02-08,"$1,872.10","$1,880.40","$1,872.10","$1,877.40","$1,877.40",230
1787,2023-02-09,"$1,875.30","$1,884.60","$1,859.80","$1,866.20","$1,866.20",223
1788,2023-02-10,"$1,861.60","$1,863.50","$1,852.40","$1,862.80","$1,862.80",27
1789,2023-02-13,"$1,859.00","$1,861.00","$1,850.00","$1,851.90","$1,851.90",569
1790,2023-02-14,"$1,854.10","$1,862.00","$1,846.20","$1,854.00","$1,854.00",569


In [15]:
# Validamos que no hay datos vacíos
df.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [17]:
# TAREA: Filtramos el dataframe para quedarnos solo las columnas "Date" y "Open"
# Date será la fecha del valor y Open el valor del BTC en ese momento
df = df[['Date','Open']]
# Valores que espera el prophet
# Eje X: Indíce llamado ds
# Eje Y: Serie a predecir llamada y
new_names = {
    "Date": "ds", 
    "Open": "y",
}

# Los renombramos con los nuevos nombres
df.rename(columns=new_names, inplace=True)
df['ds'] = df['ds'].dt.tz_localize(None)

In [18]:
df.tail()

,ds,y
1786,2023-02-08,"$1,872.10"
1787,2023-02-09,"$1,875.30"
1788,2023-02-10,"$1,861.60"
1789,2023-02-13,"$1,859.00"
1790,2023-02-14,"$1,854.10"


In [20]:
# Crear la gráfica del precio de apertura

x = df["ds"]
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Le ponemos el título
fig.update_layout(
    title_text="Serie temporal de Oro Precio de Apertura",
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1a", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

<h3 id="version">Entrenar y predecir el modelo</h3>


<p>
    Inicializamos el Prophet y lo entramos pasandole el dataframe
</p>



Documentacion oficial de Prophet [GitHub](https://facebook.github.io/prophet/docs/quick_start.html)

In [30]:
#TAREA: Inicializa Prophet en la variable m con seasonality_mode en modo "multiplicative"
m = Prophet(seasonality_mode ="multiplicative")


In [31]:
#TAREA: Entrena el modelo m con el dataframe de datos df
m.fit(df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


<p>
    Le decimos al modelo m los días en futuro que queremos predecir, en este caso el próximo año.
</p>



In [32]:
# TAREA: generar un dataframe con los días que tenemos y los 365 días siguientes a hoy para predecirlo
# PISTA: puedes usar la función del prophet make_future_dataframe()
future =  m.make_future_dataframe(periods=365)
future.tail()

,ds
2151,2024-02-10
2152,2024-02-11
2153,2024-02-12
2154,2024-02-13
2155,2024-02-14


In [40]:
# En el caso que hayas elegido un valor o fondo tradicional, es decir, que no opere los fines
# de semana, descomenta la siguiente línea:

# future = future[ future['ds'].dt.dayofweek < 5 ] # Nos elimina los fines de semana de 'ds' para no predecirlos
future = future[ future['ds'].dt.dayofweek < 5 ]

<p>
    Al modelo entrenado le pasamos el dataframe a predecir.
</p>



In [41]:
#Utiliza el modelo m para predecir el dataframe future
forecast = m.predict(future)

In [42]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2047,2024-02-08,"$1,826.37",$858.77,"$2,849.23"
2048,2024-02-09,"$1,826.96",$853.33,"$2,855.69"
2049,2024-02-12,"$1,830.55",$839.81,"$2,852.91"
2050,2024-02-13,"$1,830.44",$844.67,"$2,875.54"
2051,2024-02-14,"$1,831.71",$819.46,"$2,864.91"


In [43]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

forecast[forecast['ds'] == next_day]['yhat'].item()

1871.5348330163165

<p>
    Gráfica con el modelo entrenado y los valores de un año en futuro predecidos.
</p>



In [44]:
plot_plotly(m, forecast)

In [46]:
plot_components_plotly(m, forecast)